In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
#from datetime import timedelta  ## use this to increment targ by 1 year!!!!
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
#from patsy import dmatrices
#import random

import sys
import os
os.chdir('C:/BankRisk')

In [2]:
import time
from time import gmtime, strftime
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2017-05-20 18:05:59


In [3]:
dm = pd.read_csv("ladder_durmv_allmat_longhist.csv",header=[0,1],index_col=0,parse_dates=True) ## NOTE THIS COMMAND!!!
dm.head(2)

dv12                                                    \
                  ff        mv      mvpc       dur       cpn       mv3   
1965-12-01  4.380000  9.872790  1.008227  0.534467  5.493205  4.784516   
1966-01-01  4.481389  9.288667  1.007741  0.534508  5.463205  4.955346   

                               dv24       ...   dv300                 dv360  \
             roll12m    roll3m   mv mvpc  ...     mv3 roll300m roll3m    mv   
1965-12-01  4.854038  4.998342  NaN  NaN  ...     NaN      NaN    NaN   NaN   
1966-01-01  4.916538  5.187601  NaN  NaN  ...     NaN      NaN    NaN   NaN   

                                             
           mvpc dur cpn mv3 roll360m roll3m  
1965-12-01  NaN NaN NaN NaN      NaN    NaN  
1966-01-01  NaN NaN NaN NaN      NaN    NaN  

[2 rows x 106 columns]

In [4]:
dmq = dm.resample("Q").mean()
#dmq = dmq.dropna(how='any')
dmq.head(2)

dv12                                                   \
                  ff       mv      mvpc       dur       cpn       mv3   
1965-12-31  4.380000  9.87279  1.008227  0.534467  5.493205  4.784516   
1966-03-31  4.619954  9.60940  1.008008  0.534447  5.559872  5.331311   

                               dv24       ...   dv300                 dv360  \
             roll12m    roll3m   mv mvpc  ...     mv3 roll300m roll3m    mv   
1965-12-31  4.854038  4.998342  NaN  NaN  ...     NaN      NaN    NaN   NaN   
1966-03-31  4.986816  5.337432  NaN  NaN  ...     NaN      NaN    NaN   NaN   

                                             
           mvpc dur cpn mv3 roll360m roll3m  
1965-12-31  NaN NaN NaN NaN      NaN    NaN  
1966-03-31  NaN NaN NaN NaN      NaN    NaN  

[2 rows x 106 columns]

In [5]:
dmq120=pd.DataFrame(dmq['dv120']['roll120m'])
dmq144 = dmq['dv144']['roll144m']
dmq180 = dmq['dv180']['roll180m']
dmq240 = dmq['dv240']['roll240m']
dmq300 = dmq['dv300']['roll300m']
dmq360 = dmq['dv360']['roll360m']
dmx = pd.concat([dmq120,dmq144,dmq180,dmq240,dmq300,dmq360],axis=1)

dmx.tail(2)

roll120m  roll144m  roll180m  roll240m  roll300m  roll360m
2016-09-30  3.107139  3.488987  3.893369  4.686232   5.30785  6.033080
2016-12-31  3.019723  3.428226  3.830504  4.622741   5.24494  5.980617

In [6]:
## get libor-swap history:
ls = pd.read_csv("LibSwap_LongHist_30y_201612.csv",index_col=0)
ls.index = pd.to_datetime(ls.index)
#ls = ls[:'2014']
lsq=ls['1965':].resample('Q').mean()
## prepare to build ladders:
ld = lsq[['F1d','Prime']]
ld['DDA'] = 0.
mlst = [1,3,6,12,24,36,48,60,84,120,360]  # maturity list to be used
lcol = list(ls.columns)[1:-2]
lcol.remove('L12m')
mdct = dict(zip(mlst,lcol))
len(lsq), len(ld) # 208, 208

(208, 208)

In [7]:
mlst

[1, 3, 6, 12, 24, 36, 48, 60, 84, 120, 360]

In [8]:
## alternative code to process rolling averages:
ldm = ls[['F1d','Prime']]
m1 = ls[mdct[mlst[0]]] * 365/360
m3 = pd.rolling_mean(ls[mdct[mlst[1]]],mlst[1]) * 365/360
m6 = pd.rolling_mean(ls[mdct[mlst[2]]],mlst[2]) * 365/360
m12 = pd.rolling_mean(ls[mdct[mlst[3]]],mlst[3])
m24 = pd.rolling_mean(ls[mdct[mlst[4]]],mlst[4])
m36 = pd.rolling_mean(ls[mdct[mlst[5]]],mlst[5])
m48 = pd.rolling_mean(ls[mdct[mlst[6]]],mlst[6])
m60 = pd.rolling_mean(ls[mdct[mlst[7]]],mlst[7])
m84 = pd.rolling_mean(ls[mdct[mlst[8]]],mlst[8])
m120 = pd.rolling_mean(ls[mdct[mlst[9]]],mlst[9])
m360 = pd.rolling_mean(ls[mdct[mlst[10]]],mlst[10])
ldc = pd.concat([ldm,m1,m3,m6,m12,m24,m36,m48,m60,m84,m120,m360],axis=1)
ldc['F1d'] = ldc['F1d'] * 365/360 
ldc['Prime'] = ldc['Prime'] * 365/360 
#ldc.dropna(how='any',inplace=True)
ldx=ldc.resample('Q').mean()
#ldl=ldc.resample('Q',how='last')  # do not use
ldx.tail(4)

F1d     Prime       L1m       L3m       L6m       S1y  \
Date                                                                     
2016-03-31  0.365000  3.548611  0.436547  0.582104  0.703539  0.583611   
2016-06-30  0.378519  3.548611  0.450295  0.644753  0.873843  0.657500   
2016-09-30  0.402176  3.548611  0.514623  0.733296  0.992987  0.732778   
2016-12-31  0.456250  3.595926  0.610310  0.894356  1.169656  0.840468   

                 S2y       S3y       S4y       S5y       S7y      S10y  \
Date                                                                     
2016-03-31  0.786528  0.997870  1.164653  1.459167  2.207063  3.304611   
2016-06-30  0.834722  1.037963  1.178333  1.409222  2.153294  3.209806   
2016-09-30  0.868750  1.055185  1.199861  1.382389  2.077897  3.107139   
2016-12-31  0.916897  1.091863  1.242979  1.387110  2.019314  3.019723   

                S30y  
Date                  
2016-03-31  6.138950  
2016-06-30  6.085835  
2016-09-30  6.033080  
2016-12-31  5.980617

In [9]:
ldx = ldx['1965-12-31':]
len(ldx),len(dmx) # 205, 205

(205, 205)

In [10]:
ldx.head(2)

F1d     Prime       L1m       L3m       L6m       S1y  S2y  \
Date                                                                          
1965-12-31  4.224537  4.704444  4.611440  4.866537  4.880055  4.854038  NaN   
1966-03-31  4.619954  5.187731  5.033893  5.337432  5.256321  4.986816  NaN   

            S3y  S4y  S5y  S7y  S10y  S30y  
Date                                        
1965-12-31  NaN  NaN  NaN  NaN   NaN   NaN  
1966-03-31  NaN  NaN  NaN  NaN   NaN   NaN

In [11]:
dmx.head(2)

roll120m  roll144m  roll180m  roll240m  roll300m  roll360m
1965-12-31       NaN       NaN       NaN       NaN       NaN       NaN
1966-03-31       NaN       NaN       NaN       NaN       NaN       NaN

In [12]:
ldx[['S12y','S15y','S20y','S25y']] =dmx[['roll144m','roll180m','roll240m','roll300m']]

In [13]:
ldx = ldx[['F1d','L1m','L3m','L6m','S1y','S2y','S3y','S4y','S5y','S7y','S10y','S12y','S15y','S20y','S25y','S30y','Prime']]
ldx['DDA'] = 0
ldx.tail(2)

F1d       L1m       L3m       L6m       S1y       S2y  \
Date                                                                     
2016-09-30  0.402176  0.514623  0.733296  0.992987  0.732778  0.868750   
2016-12-31  0.456250  0.610310  0.894356  1.169656  0.840468  0.916897   

                 S3y       S4y       S5y       S7y      S10y      S12y  \
Date                                                                     
2016-09-30  1.055185  1.199861  1.382389  2.077897  3.107139  3.488987   
2016-12-31  1.091863  1.242979  1.387110  2.019314  3.019723  3.428226   

                S15y      S20y     S25y      S30y     Prime  DDA  
Date                                                              
2016-09-30  3.893369  4.686232  5.30785  6.033080  3.548611    0  
2016-12-31  3.830504  4.622741  5.24494  5.980617  3.595926    0

In [14]:
rlst = ['r'+str(x) for x in mlst+[144,180,240,300]]
rdct = dict(zip(lcol+['S12y','S15y','S20y','S25y'],rlst))
if 'L1m' in ldx.columns: ldx.rename(columns=rdct,inplace=True)
if 'F1d' in ldx.columns: ldx.rename(columns={'F1d':'r0'},inplace=True)
ldx = ldx[['r0','r1','r3','r6','r12','r24','r36','r48','r60','r84','r120','r144','r180','r240','r300','r360','Prime','DDA']]
ldx.tail()
#rdct

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2015-12-31  0.162222  0.255253  0.363004  0.530224  0.502778  0.725139   
2016-03-31  0.365000  0.436547  0.582104  0.703539  0.583611  0.786528   
2016-06-30  0.378519  0.450295  0.644753  0.873843  0.657500  0.834722   
2016-09-30  0.402176  0.514623  0.733296  0.992987  0.732778  0.868750   
2016-12-31  0.456250  0.610310  0.894356  1.169656  0.840468  0.916897   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2015-12-31  0.945278  1.143194  1.496944  2.244325  3.382139  3.682375   
2016-03-31  0.997870  1.164653  1.459167  2.207063  3.304611  3.627597   
2016-06-30  1.037963  1.178333  1.409222  2.153294  3.209806  3.562497   
2016-09-30  1.055185  1.199861  1.382389  2.077897  3.107139  3.488987   
2016-12-31  1.091863  1.242979  1.387110  2.019314  3.019723  3.428226   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2015-12-31  4.108575  4.872271  5.511094  6.202503  3.335694    0  
2016-03-31  4.039754  4.819667  5.445117  6.138950  3.548611    0  
2016-06-30  3.968053  4.755796  5.377182  6.085835  3.548611    0  
2016-09-30  3.893369  4.686232  5.307850  6.033080  3.548611    0  
2016-12-31  3.830504  4.622741  5.244940  5.980617  3.595926    0

In [15]:
mzlst = [0,1, 3, 6, 12, 24, 36, 48, 60, 84, 120]
rzlst = ['r0','r1','r3','r6','r12','r24','r36','r48','r60','r84','r120','Prime','DDA']
#if 0 not in mzlst: mzlst = [0] + mlst
#if 'r0' not in rzlst: rzlst = ['r0'] + rlst
ldxall = ldx['2001':].copy()  # keep ldxall as a reference table (for multiple runs)
ldxall.head(2)

r0        r1        r3        r6      r12       r24  \
Date                                                                    
2001-03-31  5.671019  5.584845  5.925261  6.289939  6.58615  6.441290   
2001-06-30  4.386759  4.350878  4.636757  5.156984  5.97003  6.360413   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2001-03-31  6.103628  6.175796  6.266308  6.514782  6.895917  7.321731   
2001-06-30  6.102206  6.095638  6.223161  6.481845  6.828667  7.247147   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2001-03-31  7.729644  9.055592  9.238483  9.023238  8.743102    0  
2001-06-30  7.689809  8.956643  9.214836  9.019233  7.441944    0

In [16]:
mzlst

[0, 1, 3, 6, 12, 24, 36, 48, 60, 84, 120]

In [17]:
rzlst

['r0',
 'r1',
 'r3',
 'r6',
 'r12',
 'r24',
 'r36',
 'r48',
 'r60',
 'r84',
 'r120',
 'Prime',
 'DDA']

In [18]:
ldxall.tail(2)

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2016-09-30  0.402176  0.514623  0.733296  0.992987  0.732778  0.868750   
2016-12-31  0.456250  0.610310  0.894356  1.169656  0.840468  0.916897   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2016-09-30  1.055185  1.199861  1.382389  2.077897  3.107139  3.488987   
2016-12-31  1.091863  1.242979  1.387110  2.019314  3.019723  3.428226   

                r180      r240     r300      r360     Prime  DDA  
Date                                                              
2016-09-30  3.893369  4.686232  5.30785  6.033080  3.548611    0  
2016-12-31  3.830504  4.622741  5.24494  5.980617  3.595926    0

In [19]:
grp='TestProd2'

In [20]:
## select time period: 2001-2014, 2001-2007, 2008-2014 (14y, 7y, 7y)
#r0 = '2001_2014'
#r0 = '2001_2007'
r0 = '2001_2010'  # use this for book
#r0 = '2005_2014'
ra = int(r0[0:4])
rb = int(r0[-4:])
r0, ra, rb

('2001_2010', 2001, 2010)

In [21]:
ldx = ldxall[str(ra):str(rb)]
ldx['r1.5'] = ldx['r1']*.5
ldx['r3.5'] = ldx['r3']*.5
ldx['r6.5'] = ldx['r6']*.5
ldx['r12.5'] = ldx['r12']*.5
ldx['r1.5_1'] = ldx['r1.5'].shift(1)
ldx['r3.5_1'] = ldx['r3.5'].shift(1)
ldx['r6.5_1'] = ldx['r6.5'].shift(1)
ldx['r12.5_1'] = ldx['r12.5'].shift(1)

In [22]:
ldx.head(2)

r0        r1        r3        r6      r12       r24  \
Date                                                                    
2001-03-31  5.671019  5.584845  5.925261  6.289939  6.58615  6.441290   
2001-06-30  4.386759  4.350878  4.636757  5.156984  5.97003  6.360413   

                 r36       r48       r60       r84    ...        Prime  DDA  \
Date                                                  ...                     
2001-03-31  6.103628  6.175796  6.266308  6.514782    ...     8.743102    0   
2001-06-30  6.102206  6.095638  6.223161  6.481845    ...     7.441944    0   

                r1.5      r3.5      r6.5     r12.5    r1.5_1    r3.5_1  \
Date                                                                     
2001-03-31  2.792422  2.962631  3.144970  3.293075       NaN       NaN   
2001-06-30  2.175439  2.318379  2.578492  2.985015  2.792422  2.962631   

             r6.5_1   r12.5_1  
Date                           
2001-03-31      NaN       NaN  
2001-06-30  3.14497  3.293075  

[2 rows x 26 columns]

In [23]:
ldx.tail(2)

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2010-09-30  0.189259  0.292869  0.459604  0.600520  0.593889  1.486944   
2010-12-31  0.189259  0.260921  0.302902  0.574779  0.550556  1.222222   

                 r36       r48       r60       r84    ...        Prime  DDA  \
Date                                                  ...                     
2010-09-30  2.568241  3.456181  3.955611  4.241468    ...     3.295139    0   
2010-12-31  2.265370  3.216111  3.800000  4.170159    ...     3.295139    0   

                r1.5      r3.5     r6.5     r12.5    r1.5_1    r3.5_1  \
Date                                                                    
2010-09-30  0.146434  0.229802  0.30026  0.296944  0.159510  0.178901   
2010-12-31  0.130460  0.151451  0.28739  0.275278  0.146434  0.229802   

              r6.5_1   r12.5_1  
Date                            
2010-09-30  0.239778  0.322361  
2010-12-31  0.300260  0.296944  

[2 rows x 26 columns]

In [24]:

y9col=['r1.5','r1.5_1','r3.5','r3.5_1','r6.5','r6.5_1','r12.5','r12.5_1','DDA','r6','r12']

y9col

['r1.5',
 'r1.5_1',
 'r3.5',
 'r3.5_1',
 'r6.5',
 'r6.5_1',
 'r12.5',
 'r12.5_1',
 'DDA',
 'r6',
 'r12']

In [25]:
mx = np.arange(0,1.05,.05)  # mixes to analyze
yrcol=['grp','prod','mt1','mt2','mt3','mx1','mx2','dur','spd','sd']
yr = pd.DataFrame(columns=yrcol)

In [26]:
mzlst

[0, 1, 3, 6, 12, 24, 36, 48, 60, 84, 120]

In [27]:
rzlst

['r0',
 'r1',
 'r3',
 'r6',
 'r12',
 'r24',
 'r36',
 'r48',
 'r60',
 'r84',
 'r120',
 'Prime',
 'DDA']

In [28]:
d20y = pd.read_csv('durladder_last20y_adjmnts.csv',index_col=0)
d20y.index.names = ['mnth']
d20y.reset_index(drop=False,inplace=True)
d20dct = dict(zip(d20y['mnth'],d20y['dur']))
d20dct[0] = 0
d20dct[1] = 1/24
d20dct[3] = 3/24
d20dct[6] = 6/24
d20dct

{0: 0,
 1: 0.041666666666666664,
 3: 0.125,
 6: 0.25,
 12: 0.53758767769772742,
 24: 1.0218788497143518,
 36: 1.49225182680251,
 48: 1.9470625307187936,
 60: 2.3854616812423108,
 72: 2.8069834303107006,
 84: 3.2111795033027533,
 96: 3.5979439218068587,
 108: 3.9668780457949562,
 120: 4.3212275914037672,
 144: 4.9931027561959054,
 180: 5.9029575925842419,
 240: 7.173259739520593,
 300: 8.2124566197576954,
 360: 9.0984117045783677}

In [29]:
## process set of products in ycol:

for prd in y9col:
  ldx['prd'] = ldx[prd]
  print(prd+' started at:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#  for i in [0,1]:
  for i in np.arange(0,len(mzlst)-2):
    mt1 = rzlst[i]
    for j in np.arange(i+1,len(mzlst)-1):
      mt2 = rzlst[j]
      for k in np.arange(j+1,len(mzlst)):
        mt3 = rzlst[k]
        for m1 in mx:
          for m2 in mx:
            if (m1==0)&(m2==0): sd=99999
            if (m1 + m2) <= 1:
              ldx['spd'] = ldx.apply(lambda x: x['prd'] - (m1*x[mt1] + m2*x[mt2] + (1-m1-m2)*x[mt3]),axis=1)
              if ldx['spd'].std() < sd:
                sd = ldx['spd'].std()
                avg = ldx['spd'].mean()
                dur = m1 * d20dct[mzlst[i]] + m2* d20dct[mzlst[j]] + (1-m1-m2)* d20dct[mzlst[k]]
                ydct = dict(zip(yrcol,[grp,prd,mt1,mt2,mt3,m1,m2,dur,avg,sd]))
            else:
              pass
        yr = yr.append(ydct,ignore_index=True)
    print(mt1, sd, dur, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


r1.5 started at: 2017-05-20 18:06:51
r0 0.37265831870486793 2.1606137957 2017-05-20 18:10:41
r1 0.3060964991975907 2.18144712904 2017-05-20 18:13:45
r3 0.30864949611886755 2.2231137957 2017-05-20 18:16:09
r6 0.35651725631112896 2.08205241613 2017-05-20 18:17:56
r12 0.5066922929734583 2.42940763455 2017-05-20 18:19:13
r24 0.8043277097992898 2.83652065764 2017-05-20 18:20:05
r36 0.9684408194023504 3.75543243848 2017-05-20 18:20:37
r48 0.9836023426059562 4.3212275914 2017-05-20 18:20:52
r60 0.9836023426059562 4.3212275914 2017-05-20 18:20:57
r1.5_1 started at: 2017-05-20 18:20:57
r0 0.4461658753518736 2.1606137957 2017-05-20 18:24:47
r1 0.38984441566364325 2.18144712904 2017-05-20 18:28:38
r3 0.31465220093363677 2.2231137957 2017-05-20 18:31:20
r6 0.24843666462410557 2.08205241613 2017-05-20 18:33:07
r12 0.3649864712585324 2.24022563887 2017-05-20 18:34:24
r24 0.6820202400791034 2.34161834639 2017-05-20 18:35:15
r36 0.921914171340092 3.04818849733 2017-05-20 18:35:46
r48 0.988184693803052

In [30]:
yr.head()

grp  prod mt1 mt2  mt3   mx1  mx2       dur       spd        sd
0  TestProd2  r1.5  r0  r1   r3  0.25  0.1  0.085417 -1.342089  0.907300
1  TestProd2  r1.5  r0  r1   r6  0.20  0.2  0.158333 -1.462785  0.875889
2  TestProd2  r1.5  r0  r1  r12  0.00  0.5  0.289627 -1.552709  0.856415
3  TestProd2  r1.5  r0  r1  r24  0.05  0.5  0.480679 -1.792312  0.693689
4  TestProd2  r1.5  r0  r1  r36  0.00  0.6  0.621901 -1.938803  0.511192

In [31]:
yrn = yr.ix[yr.groupby(['prod']).apply(lambda x: x['sd'].idxmin())]
yrn

grp     prod mt1  mt2   mt3   mx1   mx2       dur       spd  \
1343  TestProd2      DDA  r0   r6  r120  0.00  0.05  4.117666 -5.693406   
59    TestProd2     r1.5  r1   r6  r120  0.40  0.15  1.998719 -2.809872   
291   TestProd2   r1.5_1  r6  r48  r120  0.55  0.10  1.844636 -2.734470   
1652  TestProd2      r12  r0   r1   r12  0.00  0.00  0.537588  0.000000   
1116  TestProd2    r12.5  r6  r48  r120  0.50  0.20  1.810781 -2.574428   
1271  TestProd2  r12.5_1  r6  r24   r48  0.45  0.05  1.137125 -2.132824   
417   TestProd2     r3.5  r3   r6  r120  0.35  0.20  2.038302 -2.803781   
621   TestProd2   r3.5_1  r6  r48  r120  0.55  0.25  1.488511 -2.457996   
1486  TestProd2       r6  r0   r1    r6  0.00  0.00  0.250000  0.000000   
683   TestProd2     r6.5  r0   r6  r120  0.00  0.55  2.082052 -2.773957   
949   TestProd2   r6.5_1  r6  r48   r60  0.55  0.35  1.057518 -2.127127   

            sd  
1343  0.604397  
59    0.300484  
291   0.243864  
1652  0.000000  
1116  0.240862  
1271  0.237223  
417   0.300824  
621   0.260405  
1486  0.000000  
683   0.304578  
949   0.294148

In [32]:
yr.to_csv('yldreg_3mat_'+grp+'_'+r0+'.csv')
yrn.to_csv('yldreg_3matbest_'+grp+'_'+r0+'.csv')

In [33]:
'yldreg_3matbest_'+grp+'_'+r0+'.csv'

'yldreg_3matbest_TestProd2_2001_2010.csv'

In [34]:
len(yr), len(yrn) # 1815, 11

(1815, 11)